# Phase III: First ML Proof of Concept (5\%)

### Team Names:
- Colin Hui
- Derek Aslan
- Aydan Ali
- Conor Cummings

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# function to clean data

def clean_data(original_csv):
    """  
    Cleans the data from the original CSV file by handling missing/null values

    Args:
        original_csv (str): The path to the original CSV file we produced in Phase II.

    Returns:
        pandas.DataFrame: A cleaned DataFrame with missing/null values handled
    """

    df = pd.read_csv(original_csv)

    null_count = df.isnull().sum()
    null_pct = (null_count) / len(df) * 100

    null_df = pd.DataFrame({'null_count': null_count, 'null_pct': null_pct})
    print(null_df)

In [7]:
clean_data('world_bank_data_2023.csv')

                   null_count   null_pct
Unnamed: 0                  1   0.460829
SP.POP.TOTL                 0   0.000000
NY.GNP.PCAP.CD             22  10.138249
SI.POV.DDAY               164  75.576037
SI.POV.GINI               164  75.576037
MS.MIL.XPND.GD.ZS          69  31.797235
VA.EST                     16   7.373272
region                      0   0.000000
incomeLevel                 0   0.000000
